# Final Process

In [4]:
import pandas as pd
import concurrent.futures
import hashlib
import os
import boto3
from concurrent.futures import ThreadPoolExecutor
from tqdm import tqdm
from slugify import slugify
import datetime
import logging
import numpy as np

### Reading 7 sources csv created by us

In [5]:
brand_name='Rolex'
brand= pd.read_csv(r"C:\Users\Richard\Downloads\Bulk_Rolex_reference_number.csv")

In [6]:
brand.columns

Index(['group_reference', 'reference_number', 'watch_url', 'type', 'brand',
       'year_introduced', 'parent_model', 'specific_model', 'nickname',
       'marketing_name', 'style', 'currency', 'price', 'image_url', 'made_in',
       'case_shape', 'case_material', 'case_finish', 'caseback', 'diameter',
       'between_lugs', 'lug_to_lug', 'case_thickness', 'bezel_material',
       'bezel_color', 'crystal', 'water_resistance', 'weight', 'dial_color',
       'numerals', 'bracelet_material', 'bracelet_color', 'clasp_type',
       'movement', 'caliber', 'power_reserve', 'frequency', 'jewels',
       'features', 'description', 'short_description'],
      dtype='object')

In [4]:
#brand.to_csv('Tudor\Tudor_with_hash.csv')

In [7]:
def hash_string(url):
    # Using md5 hash function for hashing
    hashed_url = hashlib.md5(url.encode()).hexdigest()
    return hashed_url

# Apply the hash_string function to the text_column
brand['hash_watch_url'] = brand['watch_url'].apply(hash_string)



In [8]:
brand['hash_watch_url'].head(2)

0    81d8b5a380d9ee3b6337945d8bc0861c
1    1e84c4522385ffe57ad9379c3550fc1d
Name: hash_watch_url, dtype: object

In [39]:
#exporting csv with hash_watch_url
brand.to_csv(r"C:\Users\Richard\Downloads\Rolex\Bulk_Rolex_hash.csv", index=False,encoding='utf-8')

## Reading S3

In [107]:
# Initialize a boto3 S3 client
s3 = boto3.client('s3')


bucket_name = 'wcc-library'
bucket_prefix_og = 'brands_images_appended/Rolex/Original_Images/'



def list_objects_with_urls(bucket_name, bucket_prefix):
    """
    List all objects in an S3 bucket with a specific prefix and get their name and object URL.

    Parameters:
    - bucket_name (str): The name of the S3 bucket.
    - prefix (str): The prefix to filter the objects in the bucket.

    Returns:
    - A list of tuples, each containing the object's name (key) and its URL.
    """
    objects_with_urls = []

    # Paginate through returned objects in case of large result set
    paginator = s3.get_paginator('list_objects_v2')
    for page in paginator.paginate(Bucket=bucket_name, Prefix=bucket_prefix):
        if "Contents" in page:
            for obj in page['Contents']:
                key = obj['Key']
                # Construct the object URL
                object_url = f"https://{bucket_name}.s3.amazonaws.com/{key}"
                objects_with_urls.append((key, object_url))

    return objects_with_urls

# Example usage
objects_info_og = pd.DataFrame(list_objects_with_urls(bucket_name, bucket_prefix_og),columns=["s3_path","old_object_url"])

In [111]:
objects_info_og['index']=objects_info_og['s3_path'].apply(lambda x:x.split('/')[3].split('_')[0])
objects_info_og = objects_info_og[objects_info_og['index'] != '']
objects_info_og['index']=objects_info_og['index'].astype(int)
objects_info_og.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9343 entries, 1 to 9343
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   s3_path         9343 non-null   object
 1   old_object_url  9343 non-null   object
 2   index           9343 non-null   int32 
dtypes: int32(1), object(2)
memory usage: 255.5+ KB


In [113]:
objects_info_og

,s3_path,old_object_url,index
1,brands_images_appended/Rolex/Original_Images/0__Rolex__Cosmograph_Daytona_M126506-0001.png,https://wcc-library.s3.amazonaws.com/brands_images_appended/Rolex/Original_Images/0__Rolex__Cosmograph_Daytona_M126506-0001.png,0
2,brands_images_appended/Rolex/Original_Images/1000__Rolex__Datejust_36_M126281RBR-0017.png,https://wcc-library.s3.amazonaws.com/brands_images_appended/Rolex/Original_Images/1000__Rolex__Datejust_36_M126281RBR-0017.png,1000
3,brands_images_appended/Rolex/Original_Images/1001__Rolex__Datejust_36_M126234-0058.png,https://wcc-library.s3.amazonaws.com/brands_images_appended/Rolex/Original_Images/1001__Rolex__Datejust_36_M126234-0058.png,1001
4,brands_images_appended/Rolex/Original_Images/1002__Rolex__Datejust_36_M126281RBR-0016.png,https://wcc-library.s3.amazonaws.com/brands_images_appended/Rolex/Original_Images/1002__Rolex__Datejust_36_M126281RBR-0016.png,1002
5,brands_images_appended/Rolex/Original_Images/1003__Rolex__Datejust_36_M126234-0056.png,https://wcc-library.s3.amazonaws.com/brands_images_appended/Rolex/Original_Images/1003__Rolex__Datejust_36_M126234-0056.png,1003
...,...,...,...
9339,brands_images_appended/Rolex/Original_Images/997__Rolex__Datejust_36_M126233-0024.png,https://wcc-library.s3.amazonaws.com/brands_images_appended/Rolex/Original_Images/997__Rolex__Datejust_36_M126233-0024.png,997
9340,brands_images_appended/Rolex/Original_Images/998__Rolex__Datejust_36_M126281RBR-0018.png,https://wcc-library.s3.amazonaws.com/brands_images_appended/Rolex/Original_Images/998__Rolex__Datejust_36_M126281RBR-0018.png,998
9341,brands_images_appended/Rolex/Original_Images/999__Rolex__Datejust_36_M126233-0023.png,https://wcc-library.s3.amazonaws.com/brands_images_appended/Rolex/Original_Images/999__Rolex__Datejust_36_M126233-0023.png,999
9342,brands_images_appended/Rolex/Original_Images/99__Rolex__Datejust_31_M278384RBR-0039.png,https://wcc-library.s3.amazonaws.com/brands_images_appended/Rolex/Original_Images/99__Rolex__Datejust_31_M278384RBR-0039.png,99


In [114]:
objects_info_og['index'].sort_values()

1          0
1111       1
2222       2
3333       3
4444       4
        ... 
9266    9352
9267    9353
9268    9354
9269    9355
9270    9356
Name: index, Length: 9343, dtype: int32

In [139]:
brands_renamed_df = pd.merge(brand, objects_info_og, left_index=True, right_on='index', how='inner')

In [116]:
brands_renamed_df.columns

Index(['group_reference', 'reference_number', 'watch_url', 'type', 'brand',
       'year_introduced', 'parent_model', 'specific_model', 'nickname',
       'marketing_name', 'style', 'currency', 'price', 'image_url', 'made_in',
       'case_shape', 'case_material', 'case_finish', 'caseback', 'diameter',
       'between_lugs', 'lug_to_lug', 'case_thickness', 'bezel_material',
       'bezel_color', 'crystal', 'water_resistance', 'weight', 'dial_color',
       'numerals', 'bracelet_material', 'bracelet_color', 'clasp_type',
       'movement', 'caliber', 'power_reserve', 'frequency', 'jewels',
       'features', 'description', 'short_description', 'hash_watch_url',
       's3_path', 'old_object_url', 'index'],
      dtype='object')

In [117]:
brands_renamed_df.head(2)

,group_reference,reference_number,watch_url,type,brand,year_introduced,parent_model,specific_model,nickname,marketing_name,...,power_reserve,frequency,jewels,features,description,short_description,hash_watch_url,s3_path,old_object_url,index
1,126506,M126506-0001,"[""https://www.rolex.com/en-us/watches/cosmograph-daytona/m126506-0001""]","""Men's Watches""","""Rolex""",NaN,"""Cosmograph Daytona""",Cosmograph Daytona,"[""Oyster, 40 mm, platinum""]",NaN,...,"""72 Hours""",NaN,NaN,"[""Screw-down, Triplock triple waterproofness system, "", ""Highly legible Chromalight display with long-lasting blue luminescence""]",NaN,"""This Oyster Perpetual Cosmograph Daytona in platinum with an ice-blue dial and an Oyster bracelet, features a a chestnut brown Cerachrom bezel with tachymetric scale.""",81d8b5a380d9ee3b6337945d8bc0861c,brands_images_appended/Rolex/Original_Images/0__Rolex__Cosmograph_Daytona_M126506-0001.png,https://wcc-library.s3.amazonaws.com/brands_images_appended/Rolex/Original_Images/0__Rolex__Cosmograph_Daytona_M126506-0001.png,0
1111,226627,M226627-0001,"[""https://www.rolex.com/en-us/watches/yacht-master/m226627-0001""]","""Men's Watches""","""Rolex""",NaN,"""Yacht Master""",Yacht-Master 42,"[""Oyster, 42 mm, RLX titanium""]",NaN,...,"""70 Hours""",NaN,NaN,"[""Screw-down, Triplock triple waterproofness system, "", ""Highly legible Chromalight display with long-lasting blue luminescence""]",NaN,"""The Oyster Perpetual Yacht-Master 42 in RLX titanium with an intense black dial and an Oyster bracelet.""",1e84c4522385ffe57ad9379c3550fc1d,brands_images_appended/Rolex/Original_Images/1__Rolex__Yacht-Master_42_M226627-0001.png,https://wcc-library.s3.amazonaws.com/brands_images_appended/Rolex/Original_Images/1__Rolex__Yacht-Master_42_M226627-0001.png,1


In [118]:
# Please review that the merged was done correctly, s3_path should have the reference number in it
pd.set_option('display.max_colwidth', 1000)
brands_renamed_df[['reference_number','s3_path']]

,reference_number,s3_path
1,M126506-0001,brands_images_appended/Rolex/Original_Images/0__Rolex__Cosmograph_Daytona_M126506-0001.png
1111,M226627-0001,brands_images_appended/Rolex/Original_Images/1__Rolex__Yacht-Master_42_M226627-0001.png
2222,M336239-0002,brands_images_appended/Rolex/Original_Images/2__Rolex__Sky-Dweller_M336239-0002.png
3333,M124273-0001,brands_images_appended/Rolex/Original_Images/3__Rolex__Explorer_36_M124273-0001.png
4444,M126713GRNR-0001,brands_images_appended/Rolex/Original_Images/4__Rolex__GMT-Master_II_M126713GRNR-0001.png
...,...,...
9266,118138-0003,brands_images_appended/Rolex/Original_Images/9352_Rolex_Day-Date_President_in_Yellow_Gold_with_Fluted_Bezel_118138_0003_118138-0003.jpg
9267,278288rbr-0024,brands_images_appended/Rolex/Original_Images/9353_Rolex_President_31mm_in_Yellow_Gold_with_Diamond_Bezel_278288rbr_0024_278288rbr-0024.jpg
9268,128348rbr-0005,brands_images_appended/Rolex/Original_Images/9354_Rolex_Day_Date_36mm_in_Yellow_Gold_with_Diamond_Bezel_128348rbr_0005_128348rbr-0005.jpg
9269,336938-0003,brands_images_appended/Rolex/Original_Images/9355_Rolex_Sky_Dweller_42mm_in_Yellow_Gold_with_Fluted_Bezel_336938_0003_336938-0003.jpg


## Changing names

### Is important to create the folder first on the S3 bucket manually using AWS portal, in this case creating rolex inside deduplicated_data folder,then data and images folder, inside images original_images folder and resized_images folder


In [119]:
brand_name="rolex"
new_prefix=f'deduplicated_data/{brand_name}/images/original_images/'

def create_new_s3_path(row,prefix):
    
    file_extension = row['s3_path'].split('.')[-1]  # Extract the file extension
    new_key = f"{row['hash_watch_url']}.{file_extension}"  # Create new key
    new_s3_path = f"{prefix}{new_key}"  # Full new path
    return new_s3_path

# Apply the function to create the new column
brands_renamed_df['new_s3_path'] = brands_renamed_df.apply(create_new_s3_path, axis=1, prefix=new_prefix)

In [120]:
#Reviewing new path
brands_renamed_df['new_s3_path']

1       deduplicated_data/rolex/images/original_images/81d8b5a380d9ee3b6337945d8bc0861c.png
1111    deduplicated_data/rolex/images/original_images/1e84c4522385ffe57ad9379c3550fc1d.png
2222    deduplicated_data/rolex/images/original_images/621ae2d4463c6db79c4f2c5e11a67f1c.png
3333    deduplicated_data/rolex/images/original_images/e706ab8f2e40836ac3d7c7532923e5f5.png
4444    deduplicated_data/rolex/images/original_images/c9410835d4070f22e0b543060f98451f.png
                                               ...                                         
9266    deduplicated_data/rolex/images/original_images/0c20f114522706487dfed6606760b977.jpg
9267    deduplicated_data/rolex/images/original_images/da432057180543da450cb3fffee7e09c.jpg
9268    deduplicated_data/rolex/images/original_images/faf6880071ed2595fb85eb9ba479dc6c.jpg
9269    deduplicated_data/rolex/images/original_images/7fbef25f19f062d979869401d40e8b94.jpg
9270    deduplicated_data/rolex/images/original_images/e50dd7c58960546abc85c93f3

### The next code will copy the images to the new location carefully review the code before running it

In [38]:
# Setup logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Create an S3 client
s3 = boto3.client('s3')

#set name for source and destination bucket in this case is the same
source_bucket = 'wcc-library'
destination_bucket = 'wcc-library'

#this function will copy the files and change the name to the new bucket
def copy_and_rename(row):
    original_key = row.s3_path
    new_s3_path = row.new_s3_path  # Use the pre-calculated full new path
    try:
        # Copy the object to the new bucket with the new key
        s3.copy_object(
            Bucket=destination_bucket,
            CopySource={'Bucket': source_bucket, 'Key': original_key},
            Key=new_s3_path
        )
        logging.info(f"Copied and renamed from {original_key} to {new_s3_path}")
    except Exception as e:
        logging.error(f"Failed to copy and rename {original_key}: {str(e)}")

# Using ThreadPoolExecutor to handle potential large-scale operations
with ThreadPoolExecutor(max_workers=10) as executor:
    list(tqdm(executor.map(copy_and_rename, brands_renamed_df.itertuples(index=False)), total=len(brands_renamed_df)))


  0%|          | 0/9343 [00:00<?, ?it/s]2024-06-27 19:35:46,698 - INFO - Copied and renamed from brands_images_appended/Rolex/Original_Images/5__Rolex__Oyster_Perpetual_41_M124300-0001.png to deduplicated_data/rolex/images/original_images/94785ee6715bfd50306edef5ce3c43c3.png
2024-06-27 19:35:46,706 - INFO - Copied and renamed from brands_images_appended/Rolex/Original_Images/8__Rolex__Air-King_M126900-0001.png to deduplicated_data/rolex/images/original_images/bd83c3e301346363751eec5044c3af2c.png
2024-06-27 19:35:46,726 - INFO - Copied and renamed from brands_images_appended/Rolex/Original_Images/7__Rolex__Datejust_36_M126234-0051.png to deduplicated_data/rolex/images/original_images/cde0e76e15b167e8a5ef103617ee7ee1.png
2024-06-27 19:35:46,845 - INFO - Copied and renamed from brands_images_appended/Rolex/Original_Images/9__Rolex__Submariner_M124060-0001.png to deduplicated_data/rolex/images/original_images/fc82b870c05b17a057be2a3776e97bb9.png
2024-06-27 19:35:46,884 - INFO - Copied and 

## Getting new object_url

In [134]:
# Similar to before we are listing all the objects to get the object url but now for the new bucket
s3 = boto3.client('s3')

bucket_name = 'wcc-library'
new_prefix = new_prefix #new bucket were the files were copied with hash_watch_url

def list_objects_with_urls(bucket_name, bucket_prefix):
    """
    List all objects in an S3 bucket with a specific prefix and get their name and object URL.

    Parameters:
    - bucket_name (str): The name of the S3 bucket.
    - prefix (str): The prefix to filter the objects in the bucket.

    Returns:
    - A list of tuples, each containing the object's name (key) and its URL.
    """
    objects_with_urls = []

    # Paginate through returned objects in case of large result set
    paginator = s3.get_paginator('list_objects_v2')
    for page in paginator.paginate(Bucket=bucket_name, Prefix=bucket_prefix):
        if "Contents" in page:
            for obj in page['Contents']:
                key = obj['Key']
                # Construct the object URL
                object_url = f"https://{bucket_name}.s3.amazonaws.com/{key}"
                objects_with_urls.append((key, object_url))

    return objects_with_urls

# Example usage
objects_info_new = pd.DataFrame(list_objects_with_urls(bucket_name, bucket_prefix=new_prefix),columns=["s3_path","object_url"])

In [135]:
objects_info_new.head(3)

,s3_path,object_url
0,deduplicated_data/rolex/images/original_images/,https://wcc-library.s3.amazonaws.com/deduplicated_data/rolex/images/original_images/
1,deduplicated_data/rolex/images/original_images/000083afa37ff7b006ba2a681665485a.jpg,https://wcc-library.s3.amazonaws.com/deduplicated_data/rolex/images/original_images/000083afa37ff7b006ba2a681665485a.jpg
2,deduplicated_data/rolex/images/original_images/001208737e0ad6c5383a1382c854de24.png,https://wcc-library.s3.amazonaws.com/deduplicated_data/rolex/images/original_images/001208737e0ad6c5383a1382c854de24.png


In [137]:
objects_info_new['hash_watch_url']=objects_info_new['s3_path'].apply(lambda x:x.split('/')[-1].split('.')[0])
objects_info_new.head(3)

,s3_path,object_url,hash_watch_url
0,deduplicated_data/rolex/images/original_images/,https://wcc-library.s3.amazonaws.com/deduplicated_data/rolex/images/original_images/,
1,deduplicated_data/rolex/images/original_images/000083afa37ff7b006ba2a681665485a.jpg,https://wcc-library.s3.amazonaws.com/deduplicated_data/rolex/images/original_images/000083afa37ff7b006ba2a681665485a.jpg,000083afa37ff7b006ba2a681665485a
2,deduplicated_data/rolex/images/original_images/001208737e0ad6c5383a1382c854de24.png,https://wcc-library.s3.amazonaws.com/deduplicated_data/rolex/images/original_images/001208737e0ad6c5383a1382c854de24.png,001208737e0ad6c5383a1382c854de24


In [144]:
temp_df= brands_renamed_df.copy()

In [145]:
temp_df.columns

Index(['group_reference', 'reference_number', 'watch_url', 'type', 'brand',
       'year_introduced', 'parent_model', 'specific_model', 'nickname',
       'marketing_name', 'style', 'currency', 'price', 'image_url', 'made_in',
       'case_shape', 'case_material', 'case_finish', 'caseback', 'diameter',
       'between_lugs', 'lug_to_lug', 'case_thickness', 'bezel_material',
       'bezel_color', 'crystal', 'water_resistance', 'weight', 'dial_color',
       'numerals', 'bracelet_material', 'bracelet_color', 'clasp_type',
       'movement', 'caliber', 'power_reserve', 'frequency', 'jewels',
       'features', 'description', 'short_description', 'hash_watch_url',
       's3_path', 'old_object_url', 'index'],
      dtype='object')

In [146]:
#drop columns for old dataframe with old s3 path
droped_columns = ['s3_path','index','s3_path','old_object_url']
temp_df = temp_df.drop(columns=droped_columns)

In [147]:
# Select only the 'hash_watch_url' and 'object_url' columns from 'objects_info_new'
objects_info_new_selected = objects_info_new[['hash_watch_url', 'object_url']]

# Merge the DataFrames
final_df = temp_df.merge(objects_info_new_selected, on='hash_watch_url', how='left')

In [152]:
#pd.reset_option('all')
final_df

,group_reference,reference_number,watch_url,type,brand,year_introduced,parent_model,specific_model,nickname,marketing_name,...,movement,caliber,power_reserve,frequency,jewels,features,description,short_description,hash_watch_url,object_url
0,126506,M126506-0001,"[""https://www.rolex.com/en-us/watches/cosmogra...","""Men's Watches""","""Rolex""",NaN,"""Cosmograph Daytona""",Cosmograph Daytona,"[""Oyster, 40 mm, platinum""]",NaN,...,"[""Perpetual, mechanical chronograph, self-wind...",4131,"""72 Hours""",NaN,NaN,"[""Screw-down, Triplock triple waterproofness s...",NaN,"""This Oyster Perpetual Cosmograph Daytona in p...",81d8b5a380d9ee3b6337945d8bc0861c,https://wcc-library.s3.amazonaws.com/deduplica...
1,226627,M226627-0001,"[""https://www.rolex.com/en-us/watches/yacht-ma...","""Men's Watches""","""Rolex""",NaN,"""Yacht Master""",Yacht-Master 42,"[""Oyster, 42 mm, RLX titanium""]",NaN,...,"[""Perpetual, mechanical, self-winding""]",3235,"""70 Hours""",NaN,NaN,"[""Screw-down, Triplock triple waterproofness s...",NaN,"""The Oyster Perpetual Yacht-Master 42 in RLX t...",1e84c4522385ffe57ad9379c3550fc1d,https://wcc-library.s3.amazonaws.com/deduplica...
2,336239,M336239-0002,"[""https://www.rolex.com/en-us/watches/sky-dwel...","""Men's Watches""","""Rolex""",2023,"""Sky Dweller""",Sky-Dweller,"[""Oyster, 42 mm, white gold""]",NaN,...,"[""Perpetual, mechanical, self-winding, dual ti...",9002,"""72 Hours""",NaN,40.0,"[""Screw-down, Twinlock double waterproofness s...",The Rolex Sky-Dweller 336239-0002 was introduc...,"""The Oyster Perpetual Sky-Dweller in 18 kt whi...",621ae2d4463c6db79c4f2c5e11a67f1c,https://wcc-library.s3.amazonaws.com/deduplica...
3,124273,M124273-0001,"[""https://www.rolex.com/en-us/watches/explorer...","""Men's Watches""","""Rolex""",NaN,"""Explorer""",Explorer 36,"[""Oyster, 36 mm, Oystersteel and yellow gold""]",NaN,...,"[""Perpetual, mechanical, self-winding""]",3230,"""70 Hours""","28,800 A/h",31.0,"[""Screw-down, Twinlock double waterproofness s...",NaN,"""The Oyster Perpetual Explorer 36 in Oysterste...",e706ab8f2e40836ac3d7c7532923e5f5,https://wcc-library.s3.amazonaws.com/deduplica...
4,126713GRNR,M126713GRNR-0001,"[""https://www.rolex.com/en-us/watches/gmt-mast...","""Men's Watches""","""Rolex""",NaN,"""Gmt Master Ii""",GMT-Master II,"[""Oyster, 40 mm, Oystersteel and yellow gold""]",NaN,...,"[""Perpetual, mechanical, self-winding, GMT fun...",3285,"""70 Hours""",28.800 A/h,31.0,"[""Screw-down, Triplock triple waterproofness s...",NaN,"""The Oyster Perpetual GMT-Master II in Oysters...",c9410835d4070f22e0b543060f98451f,https://wcc-library.s3.amazonaws.com/deduplica...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9338,118138-0003,118138-0003,https://www.essential-watches.com/watch/Rolex-...,NaN,Rolex,NaN,President Mens 36mm,Day-Date President in Yellow Gold with Fluted ...,Day-Date President in Yellow Gold with Fluted ...,NaN,...,Automatic,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0c20f114522706487dfed6606760b977,https://wcc-library.s3.amazonaws.com/deduplica...
9339,278288rbr-0024,278288rbr-0024,https://www.essential-watches.com/watch/Rolex-...,NaN,Rolex,NaN,President Mid Size 31mm,President 31mm in Yellow Gold with Diamond Bez...,President 31mm in Yellow Gold with Diamond Bez...,NaN,...,Automatic,NaN,NaN,NaN,NaN,NaN,NaN,NaN,da432057180543da450cb3fffee7e09c,https://wcc-library.s3.amazonaws.com/deduplica...
9340,128348rbr-0005,128348rbr-0005,https://www.essential-watches.com/watch/Rolex-...,NaN,Rolex,NaN,President Mens 36mm,Day Date 36mm in Yellow Gold with Diamond Beze...,Day Date 36mm in Yellow Gold with Diamond Beze...,NaN,...,Automatic,NaN,NaN,NaN,NaN,NaN,NaN,NaN,faf6880071ed2595fb85eb9ba479dc6c,https://wcc-library.s3.amazonaws.com/deduplica...
9341,336938-0003,336938-0003,https://www.essential-watches.com/watch/Rolex-...,NaN,Rolex,NaN,Sky Dweller,Sky Dweller 42mm in Yellow Gold with Fluted Be...,Sky Dweller 42mm in Yellow Gold with Fluted Be...,NaN,...,Automatic,NaN,NaN,NaN,

In [162]:
#Reviewing how many images were copied and renamed and how many we lost, should be less than 3%
(1-final_df["watch_url"].count()/brand['watch_url'].count())*100

0.14962060489472906

In [163]:
# If this % is more than 5% please let me know
missing_images_percentage= (brand['image_url'].isna().sum()/brand['watch_url'].count())
missing_images_percentage*100

0.14962060489473122

In [164]:
#Exporting Final file
final_df.to_csv(r"C:\Users\Richard\Downloads\Rolex\Bulk_Rolex_hash.csv", index=False,encoding='utf-8')

## Downloading Bucket Images from S3 to local folder

In [165]:
# Initialize logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Initialize the S3 client
s3_client = boto3.client('s3')

def download_file(bucket_name, prefix, local_folder, object_key):
    try:
        local_file_path = os.path.join(local_folder, object_key[len(prefix):])
        os.makedirs(os.path.dirname(local_file_path), exist_ok=True)
        s3_client.download_file(bucket_name, object_key, local_file_path)
        logging.info(f'Downloaded: {object_key} to {local_file_path}')
    except Exception as e:
        logging.error(f'Error downloading {object_key}: {e}')

def download_files_concurrently(bucket_name, prefix, local_folder):
    paginator = s3_client.get_paginator('list_objects_v2')
    pages = paginator.paginate(Bucket=bucket_name, Prefix=prefix)

    # Collect all object keys
    object_keys = []
    for page in pages:
        for obj in page.get('Contents', []):
            object_keys.append(obj['Key'])

    # Download files concurrently with progress bar
    with concurrent.futures.ThreadPoolExecutor() as executor:
        with tqdm(total=len(object_keys)) as progress_bar:
            futures = {
                executor.submit(download_file, bucket_name, prefix, local_folder, key): key
                for key in object_keys
            }
            for future in concurrent.futures.as_completed(futures):
                progress_bar.update(1)
                try:
                    future.result()
                except Exception as e:
                    logging.error(f'Error: {e}')

# Set your parameters
bucket_name = 'wcc-library'
prefix = 'deduplicated_data/rolex/images/original_images/'
local_folder = 'test'  # Replace with your local folder path

# Run the download function
download_files_concurrently(bucket_name, prefix, local_folder)

  0%|          | 1/9344 [00:00<53:50,  2.89it/s]2024-06-27 21:07:31,815 - INFO - Downloaded: deduplicated_data/rolex/images/original_images/002e94720a353bae57859cbda808116a.jpg to test\002e94720a353bae57859cbda808116a.jpg
2024-06-27 21:07:31,820 - INFO - Downloaded: deduplicated_data/rolex/images/original_images/000083afa37ff7b006ba2a681665485a.jpg to test\000083afa37ff7b006ba2a681665485a.jpg
2024-06-27 21:07:31,825 - INFO - Downloaded: deduplicated_data/rolex/images/original_images/005b62f57f4fd3cf2197c7da5298e26a.png to test\005b62f57f4fd3cf2197c7da5298e26a.png
2024-06-27 21:07:31,832 - INFO - Downloaded: deduplicated_data/rolex/images/original_images/0026e7c7d4d3b2138cdb9bd777c3af6d.jpg to test\0026e7c7d4d3b2138cdb9bd777c3af6d.jpg
2024-06-27 21:07:31,853 - INFO - Downloaded: deduplicated_data/rolex/images/original_images/005f6a8b1bee008391c64c9dd9290019.jpg to test\005f6a8b1bee008391c64c9dd9290019.jpg
  0%|          | 6/9344 [00:00<09:28, 16.43it/s]2024-06-27 21:07:31,855 - INFO - D

### Final Note.
- Once that you finalized donwloading the original images with the new name to your local folder follow the guide on Notion for the image crop script that is on github.

